In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import subprocess
import time
from tabulate import tabulate
import io
import re

pd.options.mode.chained_assignment = None

# giving static date
previous_date = '2024-11-13'
calculate_date = '2024-11-14'

# taking dynamic date
# today = datetime.date.today()
# previous_date = today - datetime.timedelta(days=2)
# previous_date = previous_date.strftime("%Y-%m-%d")
# calculate_date = today - datetime.timedelta(days=1)
# calculate_date = calculate_date.strftime("%Y-%m-%d")

current_dir = os.getcwd()
directory = f'{current_dir}/Data/{calculate_date}'

out_folder = f"{current_dir}/out_folder_july/{calculate_date}"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

out_folder_incremental = f"{current_dir}/out_folder_july/{calculate_date}/incremental"
if not os.path.exists(out_folder_incremental):
    os.makedirs(out_folder_incremental)




def daily_file():
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            try:
                df = pd.read_csv(file_path, sep='\t', header=0, index_col=None)
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    #print(dfs)
    combined_df = pd.concat(dfs, ignore_index=True)
    output_file_path = f'{out_folder}/daily-formetted_{calculate_date}.csv'
    combined_df.to_csv(output_file_path, index=False)
    print(f"Combined CSV file saved to {output_file_path}")

def serch_term(patterns,directory):
    results = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                for line in content.splitlines():
                    for pattern in patterns:
                        if re.search(pattern, line):
                            results.append((file_path, line))
    split_results = []
    maximum = 0
    for file_path, line in results:
        columns = line.split('\t')
        maximum = max(maximum,len(columns))
        split_results.append([file_path] + columns)

    columns_name=[]
    for i in range(1,maximum+2):
        columns_name.append(i)

    df = pd.DataFrame(split_results,columns = columns_name).drop_duplicates()
    # if 2 in df.columns:
    #     df.drop(columns=[2],inplace=True)
    return df

# ib count calculation
def ib_count():
    global value
    previous_unique_install_base_path = f"{current_dir}/updated_total_install_base/install_base_corrected_{previous_date}.csv"
    new_install_base_path = os.path.join(current_dir, f'splunk_data/sp_{calculate_date}.csv')
    new_unique_install_base_path = os.path.join(current_dir, f'updated_total_install_base/install_base_corrected_{calculate_date}.csv')
    df = pd.read_csv(previous_unique_install_base_path)
    df = df['vin']
    df1 = pd.read_csv(new_install_base_path)
    df1 = df1['vin']
    df_con = pd.concat([df1,df],ignore_index=True).drop_duplicates()
    df_con.to_csv(new_unique_install_base_path,index=False)
    value = df_con.shape[0]
    print(value)


def credit():
    df = serch_term(cl[0],directory)
    if df.empty == False:
        df = df.drop([2],axis=1)
        df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
        df1 = df[df[3]=='addCreditCard']
        df2 = df[df[3]=='credit_card_added']
        df3 = df[df[3]=='Unable_to_add_credit_card']
        df1.rename(columns = {1:'meta',2:'track',3:'event',4:'service',5:'appname',6:'vin',7:'id1',8:'id2',9:'epoch',10:'unknown',11:'unknown2'},inplace=True)
        df2.rename(columns = {1:'meta',2:'track',3:'event',4:'service',5:'appname',6:'activity',7:'vin',8:'id1',9:'id2',10:'epoch',11:'unknown'},inplace=True)
        # df3.rename(columns = {1:'meta',2:'track',3:'event',4:'service',5:'appname',6:'vin'},inplace=True)
        df_con = pd.concat([df1,df2],axis=0)
        df_con['meta'] = calculate_date
        # df_con.to_csv(f'{out_folder}/creditcard_{calculate_date}.xlsx', mode='a', header=False, index=False)
        df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/creditcard_{previous_date}.csv")
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = pd.concat([df_previous,df_con],axis=0)
        df_con2.to_csv(f'{out_folder}/creditcard_{calculate_date}.csv', index=False)
        df_con.to_csv(f'{out_folder_incremental}/creditcard_{calculate_date}.csv', index=False)
        # df_con2.to_excel(f'{out_folder}/creditcard_{calculate_date}.xlsx',index=False)
    else:
        df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/creditcard_{previous_date}.csv")
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = df_previous
        df_con2.to_csv(f'{out_folder}/creditcard_{calculate_date}.csv', index=False)
        #df_con2.to_csv(f'{out_folder}/purchase-combined_{calculate_date}.csv',index=False)



def purchase():
    df = serch_term(pl[0],directory)
    if df.empty == False:
        df = df.drop([2],axis=1)
        df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
        df1 = df[df[3]=='purchaseSuccessful']
        df2 = df[df[3]=='purchaseError']
        df3 = df[df[3]=='startPurchase']
        df1.rename(columns = {1:'DateAndFilename',3:'status',4:'U1',5:'Package',6:'U2', 7:'U6',8:'VIN',9:'U3',10:'U4',11:'Timestamp',12:'U5'},inplace=True)
        df2.rename(columns = {1:'DateAndFilename',3:'status',4:'U1',5:'Package',6:'U2', 7:'VIN',8:'U3',9:'U4',10:'Timestamp',11:'U5',12:'U6'},inplace=True)
        df3.rename(columns = {1:'DateAndFilename',3:'status',4:'U1',5:'Package',6:'U2', 7:'VIN',8:'U3',9:'U4',10:'Timestamp',11:'U5',12:'U6'},inplace=True)
        df_con = pd.concat([df3,df2,df1],axis=0)
        df_con['DateAndFilename'] = calculate_date 
        df_con.to_csv(f'{out_folder_incremental}/purchase-combined_{calculate_date}.csv', index=False)
        df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/purchase-combined_{previous_date}.csv")
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = pd.concat([df_previous,df_con],axis=0)
        df_con2.to_csv(f'{out_folder}/purchase-combined_{calculate_date}.csv',index=False)
    else:
        df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/purchase-combined_{previous_date}.csv")
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = df_previous
        df_con2.to_csv(f'{out_folder}/purchase-combined_{calculate_date}.csv',index=False)


def network():
    df = serch_term(nl[0],directory)
    if df.empty == False:
        df = df.drop([2],axis=1)
        df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
        df1 = df[df[11].isnull()]
        df2 = df[df[11].notna()]
        df1.rename(columns = {1:'meta',3:'event',4:'appname',5:'network',6:'vin',7:'id1',8:'id2',9:'epoch',10:'unknown',11:'service'},inplace=True)
        df1 = df1.reindex(columns=['meta','event','service','appname','network','vin','id1','id2','epoch','unknown'])
        df2.rename(columns = {1:'meta',3:'event',4:'service',5:'appname',6:'network',7:'vin',8:'id1',9:'id2',10:'epoch',11:'unknown'},inplace=True)
        df_con = pd.concat([df2,df1],axis=0)
        df_con['meta'] = calculate_date 
        df_con.to_csv(f'{out_folder_incremental}/network_{calculate_date}.csv', index=False)
        df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/network_{previous_date}.csv")
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = pd.concat([df_previous,df_con],axis=0)
        df_con2.to_csv(f'{out_folder}/network_{calculate_date}.csv',index=False)
        # df_con.to_excel(f'{out_folder}/purchase-combined_{calculate_date}.xlsx',index=False)
    else:
        df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/network_{previous_date}.csv")
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = df_previous
        df_con2.to_csv(f'{out_folder}/purchase-combined_{calculate_date}.csv',index=False)


       

def report():
    df = serch_term(rl[0],directory)
    if df.empty == False:
        df = df.drop([2],axis=1)
        df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
        df = df[df[3] == 'applaunchmode']
        df1 = df[df[12].notna()]
        df2 = df[df[12].isnull()]
        df1.drop([1,3,4,9,10,12,13,14,15,16], axis=1, inplace=True)
        df2.drop([1,3,8,9,11,12,13,14,15,16], axis=1, inplace=True)
        df1 = df1.rename(columns={5: 'AppTray',6:'Shortcut',7:'Desktop',8:'Vin',11:'Epoch'})
        df2 = df2.rename(columns={4: 'AppTray',5:'Shortcut',6:'Desktop',7:'Vin',10:'Epoch'})
        # df2 = df2.reindex(columns=['Vin', 'AppTray', 'Shortcut','Desktop','Epoch'])
        df_con = pd.concat([df1, df2], ignore_index=True)

        df_con['AppTray'] = pd.to_numeric(df_con['AppTray'], errors='coerce')
        df_con = df_con.dropna(subset=['AppTray'])

        df_con['AppTray'] = df_con['AppTray'].astype(int)
        df_con['Shortcut'] = df_con['Shortcut'].astype(int)
        df_con['Desktop'] = df_con['Desktop'].astype(int)
        df_con['Epoch'] = df_con['Epoch'].astype(int)
        df_con.insert(0,"Time",df_con.apply(lambda df: datetime.datetime.fromtimestamp(df["Epoch"] / 1000).strftime("%m/%d/%Y %H:%M:%S"), axis=1))
        df_con = df_con[["Vin","AppTray","Shortcut","Desktop"]]
        df_con.to_csv(f'{out_folder_incremental}/out_{calculate_date}.csv', index=False)
        df_previous = pd.read_csv(f'{current_dir}/out_folder_july/{previous_date}/out_{previous_date}.csv')
        if 'Unnamed: 0' in df_previous.columns:
            df_previous.drop(columns=['Unnamed: 0'],inplace=True)
        df_con2 = pd.concat([df_previous,df_con],axis=0)
        df_con2.to_csv(f'{out_folder}/out_{calculate_date}.csv',index=False)

        df7 = df_con2.groupby('Vin', as_index=False).agg({'AppTray': 'max', 'Shortcut': 'max', 'Desktop': 'max'})
        num = df7['Vin'].count()
        count0 =  df7[df7['Shortcut'] == 0]['Shortcut'].count()
        count1 =  df7[df7['Shortcut'] == 1]['Shortcut'].count()
        count2 =  df7[df7['Shortcut'] == 2]['Shortcut'].count()
        count3 =  df7[df7['Shortcut'] == 3]['Shortcut'].count()
        count4 =  df7[df7['Shortcut'] > 3]['Shortcut'].count()
        percentage = round((num-count0)*100/value,2)
        print("VINS interacted with IVWW: ", num)
        print(f"Discovery % of {value} vehicle Install Base: ", percentage, "%\n\n")
        counts_df = pd.DataFrame({
            'Category': ['Never Clicked Discovery', 'Clicked Discovery Once', 'Clicked Discovery Twice', 'Clicked Discovery Thrice', 'Clicked Discovery more than 3'],
            'Count': [count0,count1,count2,count3,count4],
            'Percentage (%)':[round(count0 * 100 / num,2),round(count1 * 100 / num,2),round(count2 * 100 / num,2),round(count3 * 100 / num,2),round(count4 * 100 / num,2)],
        })
        def format(df):
            return (tabulate(df, headers = 'keys', tablefmt = 'psql'))
        var = format(counts_df)

        # used to generate report.txt
        f = open(f"{out_folder}/report_output.txt", "w")
        f.write(f"VINS interacted with IVWW:  {num}")
        f.write("\n")
        f.write(f"Discovery % of {value} vehicle Install Base:  {percentage} %")
        f.write("\n\n")
        
        f.write(f"{var}")
        f.close()
        print(var)
        
    
cl = [[r'addCreditCard', r'credit_card_added'],[1,2,3,4,5,6,7,8,9,10]]
pl = [[r'startPurchase', r'purchaseSuccessful', r'purchaseError'],[1,2,3,4,5,6,7,8,9,10,11,12]]
nl =[[r'4G Wi-Fi', r'SIRIUSXM', r'SiriusXM Radio', r'Wi-Fi', r'Radio SiriusXM', r'Wi-Fi Hotspot'],[1,2,3,4,5,6,7,8,9,10,11]]
rl = ['applaunchmode',[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]]


t1 = time.time()
daily_file()
ib_count()
credit()
purchase()
network()
report()
t2 = time.time()
print(t2-t1)


Combined CSV file saved to /home/t0206qb/Documents/Report/out_folder_july/2024-11-14/daily-formetted_2024-11-14.csv
1496421


/tmp/ipykernel_8637/3753798278.py:101: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
/tmp/ipykernel_8637/3753798278.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
/tmp/ipykernel_8637/3753798278.py:159: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
/tmp/ipykernel_8637/3753798278.py:168: DtypeWarning: Columns (8,9,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_previous = pd.read_csv(f"{current_dir}/out_folder_july/{previous_date}/network_{previous_date}.csv")
/tmp/ipykernel_8637/3753798278.py:188: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda 

VINS interacted with IVWW:  1002417
Discovery % of 1496421 vehicle Install Base:  66.34 %


+----+-------------------------------+---------+------------------+
|    | Category                      |   Count |   Percentage (%) |
|----+-------------------------------+---------+------------------|
|  0 | Never Clicked Discovery       |    9715 |             0.97 |
|  1 | Clicked Discovery Once        |  418224 |            41.72 |
|  2 | Clicked Discovery Twice       |  235039 |            23.45 |
|  3 | Clicked Discovery Thrice      |  140192 |            13.99 |
|  4 | Clicked Discovery more than 3 |  199247 |            19.88 |
+----+-------------------------------+---------+------------------+
104.15223717689514


In [2]:

!pip install openpyxl
!pip install "numpy<2.0"

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [3]:
pip install pandas

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [6]:
pip install pandas

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai